# Netflix and IMDB Data Analysis Project – Data Preparation

Project Overview


Data Sources
- IMDB datasets （title.basics.tsv, title.ratings.tsv）
- Netflix dataset（netflix_titles.csv）

In [1]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
%matplotlib inline


plt.rcParams['font.sans-serif'] = ['Arial Unicode MS']
basics_df = pd.read_csv('data/title.basics.tsv', sep='\t', low_memory=False)
ratings_df = pd.read_csv('data/title.ratings.tsv', sep='\t')
netflix_df = pd.read_csv('data/netflix_titles.csv')


print("IMDB basics", basics_df.shape)
print("IMDB rating:", ratings_df.shape)
print("Netflix:", netflix_df.shape)

IMDB basics (11396577, 9)
IMDB rating: (1525669, 3)
Netflix: (8807, 12)


## Data Exploration

### 1. Check Basic Information of Datasets
We first need to check the basic information of the dataset, including its structure, column names, data types, sample rows, and any duplicate entries, to ensure the data is ready for analysis.

In [2]:
# Print column names
print("\nColumn names:")
print(basics_df.columns.tolist())

# Print data types
print("\nData types:")
print(basics_df.dtypes)

# Print the first 5 rows of data
print("\nFirst 5 rows of data:")
print(basics_df.head())

# Check for duplicates in the 'tconst' column
print("\n=== Checking for duplicate tconst entries ===")
duplicates_count = basics_df['tconst'].duplicated().sum()

if duplicates_count > 0:
    print(f"\nFound {duplicates_count} duplicate tconst entries. Removing duplicates...")
    basics_df = basics_df.drop_duplicates(subset='tconst', keep='first')
    print("Duplicates have been removed.")
else:
    print("\nNo duplicate tconst entries found.")




Column names:
['tconst', 'titleType', 'primaryTitle', 'originalTitle', 'isAdult', 'startYear', 'endYear', 'runtimeMinutes', 'genres']

Data types:
tconst            object
titleType         object
primaryTitle      object
originalTitle     object
isAdult           object
startYear         object
endYear           object
runtimeMinutes    object
genres            object
dtype: object

First 5 rows of data:
      tconst titleType            primaryTitle           originalTitle  \
0  tt0000001     short              Carmencita              Carmencita   
1  tt0000002     short  Le clown et ses chiens  Le clown et ses chiens   
2  tt0000003     short            Poor Pierrot          Pauvre Pierrot   
3  tt0000004     short             Un bon bock             Un bon bock   
4  tt0000005     short        Blacksmith Scene        Blacksmith Scene   

  isAdult startYear endYear runtimeMinutes                    genres  
0       0      1894      \N              1         Documentary,Short  
1  

### 2. Data Cleaning and Preparation
Then, we merge datasets, clean the data, and rename necessary columns to ensure consistency and usability for further analysis. 


In [3]:
imdb_df = pd.merge(basics_df, ratings_df, on='tconst', how='inner')
imdb_df['startYear'] = pd.to_numeric(imdb_df['startYear'], errors='coerce')
imdb_df = imdb_df[(imdb_df['startYear'] >= 1925) & (imdb_df['startYear'] <= 2021)]


imdb_df['title_lower'] = imdb_df['primaryTitle'].str.lower().str.strip()
netflix_df['title_lower'] = netflix_df['title'].str.lower().str.strip()
netflix_df['release_year'] = pd.to_numeric(netflix_df['release_year'], errors='coerce')


matched_df = pd.merge(
    imdb_df,
    netflix_df,
    left_on=['title_lower', 'startYear'],
    right_on=['title_lower', 'release_year'],
    how='inner'
)

The column name 'cast' has been renamed to 'crew' because 'cast' is a reserved keyword in SQL. 

In [4]:

final_columns = {
    'tconst': 'movie_id',
    'primaryTitle': 'title',
    'startYear': 'release_year',
    'genres': 'imdb_genres',
    'averageRating': 'imdb_rating',
    'numVotes': 'imdb_votes',
    'show_id': 'netflix_id',
    'type': 'netflix_type',
    'director': 'director',
    'cast': 'crew',           # Rename 'cast' to 'crew'
    'country': 'country',
    'duration': 'netflix_duration',
    'listed_in': 'netflix_genres'
}


In the final step, we export the dataset as a CSV file to ensure it is stored in a structured format, making it easy for further analysis.

In [5]:
final_df = matched_df[final_columns.keys()].rename(columns=final_columns)

final_df.to_csv('/Users/mimi/Documents/imdb-analysis/data/netflix_imdb_matched.csv', 
                index=False, sep='\t', encoding='utf-8')


import subprocess
subprocess.run(['jupyter', 'nbconvert', '--to', 'html',
               '--output', 'data_preparation_report.html',
               'data_preparation.ipynb'], stderr=subprocess.STDOUT)

print("\nNotebook exported as HTML: data_preparation_report.html")

[NbConvertApp] Converting notebook data_preparation.ipynb to html
[NbConvertApp] Writing 290395 bytes to data_preparation_report.html

Notebook exported as HTML: data_preparation_report.html
